In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
data = pd.read_excel('정제파일.xlsx')
column_name='제목_초록'
cv = CountVectorizer(max_features=99999, ngram_range=(1, 1))
tdm = cv.fit_transform(data[column_name])
word_count_tf = pd.DataFrame({'단어': cv.get_feature_names_out(), '빈도': tdm.sum(axis=0).flat})
word_count_tf = word_count_tf.sort_values('빈도', ascending=False)
word_count_tf = word_count_tf.reset_index(drop=True)
word_count_tf.index = word_count_tf.index + 1

In [2]:
word_count_tf=word_count_tf.head(30)

In [6]:
word_count_tf.to_excel("빈도상위30위.xlsx",index=False)

In [7]:
# 상위 빈도 N개 기준
from itertools import combinations

# 상위 30개 단어 추출
top_30_words = word_count_tf['단어'].head(30).tolist()
# 리뷰 데이터에서 자주 등장하는 단어 30개를 리스트로 추출

# 원본 리뷰 데이터 불러오기
data = data.astype('str')
# 데이터의 모든 값을 문자열로 변환

# '[' ']', ',' 그리고 "'"를 삭제하는 코드
chars_to_remove = r"[\[\]',]"
data = data.replace(chars_to_remove, '', regex=True)
# 텍스트 데이터에서 불필요한 문자들을 제거하여 분석 준비

# 동시 출현 빈도 계산을 위한 빈 딕셔너리 생성
co_occurrence = {word: {other_word: 0 for other_word in top_30_words} for word in top_30_words}
# 각 단어 쌍의 동시 출현 빈도를 저장할 빈 딕셔너리 생성

# 각 리뷰에 대해 동시 출현 빈도 계산
for review in data[column_name]:
    # 리뷰에서 상위 30개 단어 찾기
    words_in_review = set(word for word in review.split() if word in top_30_words)
    # 리뷰에서 상위 30개 단어만 골라 세트로 만듦

    # 단어 쌍에 대해 동시 출현 빈도 증가
    for word1, word2 in combinations(words_in_review, 2):
        co_occurrence[word1][word2] += 1
        co_occurrence[word2][word1] += 1
    # 단어 쌍이 함께 등장할 때마다 해당 쌍의 빈도수를 1씩 증가

# 동시 출현 매트릭스를 DataFrame으로 변환
co_occurrence_df = pd.DataFrame(co_occurrence)
# 동시 출현 빈도를 데이터프레임으로 변환하여 표 형태로 정리

# 결과를 CSV 파일로 저장
co_occurrence_df.to_excel("1_mode_matrix.xlsx")
# 결과를 CSV 파일로 저장하여 나중에 분석할 수 있도록 함

co_occurrence_df.head()  # 결과의 첫 부분만 보여줌
# 결과의 첫 몇 줄을 화면에 출력하여 확인


,지능,인공,윤리,연구,교육,대한,인간,데이터,기술,문제,...,학습,논문,통해,제시,과정,관계,관점,편향,중심,관련
지능,0,49,45,35,22,35,25,15,22,26,...,8,14,18,18,17,13,14,11,20,18
인공,49,0,45,35,22,35,25,15,22,26,...,8,14,18,18,17,13,14,11,20,18
윤리,45,45,0,34,20,32,24,14,22,26,...,7,13,17,18,16,12,12,11,17,17
연구,35,35,34,0,21,27,19,13,20,23,...,8,8,16,17,13,10,10,6,16,18
교육,22,22,20,21,0,15,12,5,15,11,...,6,5,6,8,7,8,7,4,11,11


In [18]:
# 선택단어 기준
# from itertools import combinations
# data = data.astype('str')
# chars_to_remove = r"[\[\]',]"
# data = data.replace(chars_to_remove, '', regex=True)
# my_selected_words = ['단어1', '단어2', '단어3', ...]  # 내가 선택한 단어 리스트
# # 동시 출현 빈도 계산을 위한 빈 딕셔너리 생성
# co_occurrence = {word: {other_word: 0 for other_word in my_selected_words} for word in my_selected_words}
# # 각 리뷰에 대해 동시 출현 빈도 계산
# for review in data[column_name]:
#     # 리뷰에서 내가 선택한 단어 찾기
#     words_in_review = set(word for word in review.split() if word in my_selected_words)
#     # 단어 쌍에 대해 동시 출현 빈도 증가
#     for word1, word2 in combinations(words_in_review, 2):
#         co_occurrence[word1][word2] += 1
#         co_occurrence[word2][word1] += 1
# # 동시 출현 매트릭스를 DataFrame으로 변환
# co_occurrence_df_seleteced = pd.DataFrame(co_occurrence)
# co_occurrence_df_seleteced.to_csv(f"{filename}_1_mode_matrix_selected.csv",encoding='cp949')
# co_occurrence_df_seleteced.head()  # 결과의 첫 부분만 보여줌